# Creating a basic Cluster Expansion

In [1]:
import random
import numpy as np
from monty.serialization import loadfn, dumpfn
from pymatgen.core.structure import Structure
from smol.cofe import ClusterSubspace, StructureWrangler, ClusterExpansion, RegressionData

No module named 'phonopy'
No module named 'phonopy'


In [5]:
# load the prim structure
prim_path = '/Users/myless/Dropbox (MIT)/Research/2024/Spring_2024/Computation/structure_maker/v4cr4ti_prim_cell.json'
lno_prim = loadfn(prim_path)
    
# load the fitting data
entry_path = '/Users/myless/Dropbox (MIT)/Research/2024/Spring_2024/Computation/structure_maker/vcrti_fixed_entries.json'
lno_entries = loadfn(entry_path)

### 0) The prim structure
The prim structure defines the **configurational space** for the Cluster Expansion. 
The **configurational space** is defined by the site **compositional spaces** and the crystal symetries of the prim structure.
The occupancy of the sites determine site **compositional spaces**. Sites are **active** if they have compositional degrees of freedom.


Active sites have fractional compositions. Vacancies are allowed in sites where the composition does not sum to one.

0. Is active. The allowed species are: Li+ and vacancies.
1. Is active. The allowed species are: Ni3+ and Ni4+.
2. Is not active. Only O2- is allowed.
3. Is not active. Only O2- is allowed.

In [6]:
print(lno_prim)

Full Formula (Ti0.09375 V1.8125 Cr0.09375)
Reduced Formula: Ti0.09375V1.8125Cr0.09375
abc   :   3.010000   3.010000   3.010000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (2)
  #  SP                             a    b    c
---  ---------------------------  ---  ---  ---
  0  Ti:0.047, V:0.906, Cr:0.047  0    0    0
  1  Ti:0.047, V:0.906, Cr:0.047  0.5  0.5  0.5


### 1) The cluster subspace
The `ClusterSubspace` represents all the orbits (groups of equivalent clusters) that will be considered when fitting the cluster expansion. Its main purpose is to compute the **correlations functions** for each included orbit given a structure in the compositional space defined by the prim.

In order to do be able to compute the correlation functions, the given structure must match the prim structure in a "crystallographic" sense but allowing for compositional degrees of freedom in the "active" sites.

A cluster subspace most easily created by providing:
1. The prim structure representing the configurational space.
2. A set of diameter cutoffs for each size of orbit we want to consider.
3. A type of site basis function to use.

There are more options allowed by the code to fine grain and tune. See other notebooks for advanced use cases.

In [7]:
subspace = ClusterSubspace.from_cutoffs(
    lno_prim,
    cutoffs={2: 5, 3: 4.1}, # will include orbits of 2 and 3 sites.
    basis='sinusoid', # sets the site basis type, default is indicator
    supercell_size='num_sites'
)

# supercell_size specifies the method to determine the supercell size
# when trying to match a structure.
# (See pymatgen.structure_matcher.StructureMatcher for more info)

print(subspace) # single site and empty orbits are always included.

Basis/Orthogonal/Orthonormal : sinusoid/True/False
       Unit Cell Composition : V1.8125 Cr0.09375 Ti0.09375
            Number of Orbits : 7
No. of Correlation Functions : 21
             Cluster Cutoffs : 2: 4.99, 3: 3.01
              External Terms : []
Orbit Summary
 ------------------------------------------------------------------------
 |  ID     Degree    Cluster Diameter    Multiplicity    No. Functions  |
 |   0       0             NA                 0                1        |
 |   1       1            0.0000              2                2        |
 |   2       2            2.6067              8                3        |
 |   3       2            3.0100              6                3        |
 |   4       2            4.2568              12               3        |
 |   5       2            4.9915              24               3        |
 |   6       3            3.0100              24               6        |
 ------------------------------------------------------------

#### 1.1) Computing a correlation vector.
A correlation vector for a specific structure (represents the feature vector) used to train and predict target values.

In [8]:
structure = lno_entries[1].structure
corr = subspace.corr_from_structure(structure)

print(f'The correlation vector for a structure with'
      f' composition {structure.composition} is: '
      f'\n{corr}')

The correlation vector for a structure with composition Cr3 V58 Ti3 is: 
[ 1.          0.4296875  -0.74424058  0.1796875  -0.32137661  0.55078125
  0.1796875  -0.3179937   0.5625      0.18554688 -0.3179937   0.54492188
  0.18554688 -0.31968516  0.5546875   0.07226562 -0.13362501 -0.13531647
  0.23730469  0.24023437 -0.41271523]


### 2) The structure wrangler
The `StructureWrangler` is a class that will is used to create and organize the data that will be used to train (and possibly test) the cluster expansion. It makes sure that all the supplied structures appropriately match the prim structure, and obtains the necessary information to correctly normalize target properties (such as energy) necessary for training.

Training data is added to a `StructureWrangler` using `ComputedStructureEntry` instances from `pymatgen`.

Matching relaxed structures can be a tricky problem, especially for ionic systems with vacancies. See the notebook on structure matching for tips on how to tweak parameters.

In [9]:
wrangler = StructureWrangler(subspace)

# the energy is taken directly from the ComputedStructureEntry
# any additional properties can also be added, see notebook on
# training data preparation for an example.
for entry in lno_entries:
    wrangler.add_entry(entry, verbose=True)
# The verbose flag will print structures that fail to match.

print(f'\nTotal structures that match {wrangler.num_structures}/{len(lno_entries)}')


Total structures that match 30/30


## 3) Training

Training a cluster expansion is one of the most critical steps. This is how you get **effective cluster interactions (ECI's)**. To do so you need an estimator class that implements some form of regression model. In this case we will use simple least squares regression using the `LinearRegression` estimator from `scikit-learn`.

In `smol` the coefficients from the fit are not exactly the ECI's but the ECI times the multiplicity of their orbit.

In [10]:
from sklearn.linear_model import LinearRegression
# Set fit_intercept to False because we already do this using
# the empty cluster.
estimator = LinearRegression(fit_intercept=False)
estimator.fit(wrangler.feature_matrix, wrangler.get_property_vector('energy'))
coefs = estimator.coef_

#### 3.1) Check the quality of the fit
There are many ways to evaluate the quality of a fit. The simplest involve stadard training set prediction error metrics. But when evaluating a CE more seriously we need to consider further metrics and how the CE will be used.
Here we will just look at in sample mean squared error and max error.

In [11]:
from sklearn.metrics import mean_squared_error, max_error

train_predictions = np.dot(wrangler.feature_matrix, coefs)

rmse = mean_squared_error(
    wrangler.get_property_vector('energy'), train_predictions, squared=False
)
maxer = max_error(wrangler.get_property_vector('energy'), train_predictions)

print(f'RMSE {1E3 * rmse} meV/prim')
print(f'MAX {1E3 * maxer} meV/prim')

RMSE 0.125699734784017 meV/prim
MAX 0.33066046893992507 meV/prim


/opt/homebrew/Caskroom/miniforge/base/envs/smol/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### 4) The cluster expansion
Now we can use the above work to create the `ClusterExpansion`. The cluster expansion can be used to predict the fitted property for new structures, either for testing quality or for simulations such as in Monte Carlo.
Note that when using the `predict` function, the cluster expansion will have to match the given structure if it has not seen it before.
We will also store the details of the regression model used to fit the cluster expansion by using a `RegressionData` object.

In [12]:
reg_data = RegressionData.from_sklearn(
    estimator, wrangler.feature_matrix,
    wrangler.get_property_vector('energy')
)


expansion = ClusterExpansion(
    subspace, coefficients=coefs, regression_data=reg_data
)

structure = random.choice(wrangler.structures)
prediction = expansion.predict(structure, normalized=True)

print(
    f'The predicted energy for a structure with composition '
    f'{structure.composition} is {prediction} eV/prim.\n'
)
print(f'The fitted coefficients are:\n{expansion.coefs}\n')
print(f'The effective cluster interactions are:\n{expansion.eci}\n')
print(expansion)

The predicted energy for a structure with composition V58 Ti3 Cr3 is -17.819349896464626 eV/prim.

The fitted coefficients are:
[-1.01826989e+01 -4.37537843e+00  7.57837775e+00 -9.22415845e-01
  3.86165493e-02  1.53056130e-01 -5.00751375e-01 -6.27154603e-03
  4.84660551e-02 -1.28216469e-01 -3.55059591e-02  1.51277609e-03
 -3.39008406e-01 -3.76788231e-01 -7.96106246e-02  8.35904929e-01
 -7.22871861e-02  9.65403884e-02  6.40630363e-02 -5.39126514e-02
  5.59509575e-02]

The effective cluster interactions are:
[-1.01826989e+01 -2.18768922e+00  3.78918887e+00 -1.15301981e-01
  2.41353433e-03  1.91320162e-02 -8.34585626e-02 -5.22628836e-04
  8.07767585e-03 -1.06847058e-02 -1.47941496e-03  1.26064674e-04
 -1.41253503e-02 -7.84975481e-03 -3.31710936e-03  3.48293720e-02
 -1.50598304e-03  4.02251618e-03  1.33464659e-03 -2.24636047e-03
  2.33128990e-03]

Basis/Orthogonal/Orthonormal : sinusoid/True/False
       Unit Cell Composition : V1.8125 Cr0.09375 Ti0.09375
            Number of Orbits : 7
N

### 5) Saving your work
All core classes in `smol` are `MSONables` and so can be saved using their `as_dict` methods or better yet with `monty.serialization.dumpfn`.

Currently there is also a convenience function in `smol` that will nicely save all of your work for you in a standardized way. Work saved with the `save_work` function is saved as a dictionary with standardized names for the classes. Since a work flow should only contain 1 of each core classes the function will complain if you give it two of the same class (i.e. two wranglers)

In [13]:
from smol.io import save_work

file_path = 'v4cr4ti_fin_work.mson'
# we can save the subspace as well, but since both the wrangler
# and the expansion have it, there is no need to do so.
save_work(file_path, wrangler, expansion)

#### 5.1) Loading previously saved work

In [11]:
from smol.io import load_work

work = load_work(file_path)
for name, obj in work.items():
    print(f'{name}: {type(obj)}\n')

StructureWrangler: <class 'smol.cofe.wrangling.wrangler.StructureWrangler'>

ClusterExpansion: <class 'smol.cofe.expansion.ClusterExpansion'>

